In [57]:
import numpy as np
import cv2
from os import listdir
from matplotlib import pyplot as plt
import PIL.Image as pil
import pandas as pd

In [49]:
# Import custom methods
import general_methods
import semantic_segmentation

In [50]:
# Get pre-trained models
seg_model = semantic_segmentation.get_pretrained_model()

In [51]:
# Method to perform segmentation and return black image with only walls in grey

def wall_segmentation(filename):
    input_pil = general_methods.import_and_resize(filename)
    input_img = general_methods.import_mx_image(filename)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    size = (width, height)

    # Segmentation
    mmask = semantic_segmentation.get_segementation(input_img, seg_model)
    labels = general_methods.get_labels_string(mmask, mmask.shape[0])
    walls = general_methods.find_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    black = np.zeros((height, width, 3))
    segmented_input = semantic_segmentation.remove_inds(black, [walls], [120, 120, 120])

    return segmented_input

In [ ]:
# Change seg images so only walls are grey

folder_dir = "images/seg-images/"
 
for image in listdir(folder_dir):
    filename = "images/seg-images/" + image
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    img = input_cv2.copy()
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    for i in range(height):
        for j in range(width):
            if img[i][j][0] == 0:
                img[i][j] = [0, 0, 0]
            if img[i][j][0] != 0:
                img[i][j] = [120, 120, 120]
    
    cv2.imwrite("images/seg-images-after/" + image, img) 

In [ ]:
for image in listdir("data-collection/groundtruths/segmentation/"):
    filename = "data-collection/groundtruths/segmentation/" + image
    image_name = image[:-13]
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]

    input_pil = pil.open("images/outputs/segmentation/" + image_name + ".jpg")
    input_pil = input_pil.resize((width, height))
    input_pil.save("images/outputs/segmentation-resized/" + image_name + ".jpg")

    input_pil = pil.open("images/seg-images/" + image_name + ".png")
    input_pil = input_pil.resize((width, height))
    input_pil.save("images/seg-images-resized/" + image_name + ".png")

ADE_val_00000079_wall_seg.png


In [ ]:
for image in listdir("images/outputs/segmentation/"):
    filename = "images/outputs/segmentation/" + image
    image_name = image[:-4]
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]

    try:
        img1 = cv2.imread("images/seg-images/" + image_name + ".png")
        img1 = cv2.resize(img1,(width, height),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
        cv2.imwrite("images/seg-images-resized/" + image_name + ".png", img1)
    except:
        print(image_name + " doesn't exist")

    img2 = cv2.imread("data-collection/groundtruths/segmentation/" + image_name + "_wall_seg.png")
    img2 = cv2.resize(img2,(width, height),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
    cv2.imwrite("data-collection/groundtruths/segmentation-resized/" + image_name + "_wall_seg.png", img2)

## Intersection over Union

In [54]:
mine = general_methods.import_cv2_image("images/outputs/segmentation/ADE_val_00000079.jpg")
wiz = general_methods.import_cv2_image("images/seg-images-after/ADE_val_00000079.png")
gt = general_methods.import_cv2_image("data-collection/groundtruths/segmentation-resized/ADE_val_00000079_wall_seg.png")

In [60]:
def get_iuo(image_1, image_2):
    # https://towardsdatascience.com/intersection-over-union-iou-calculation-for-evaluating-an-image-segmentation-model-8b22e2e84686
    intersection = np.logical_and(image_1, image_2)
    union = np.logical_or(image_1, image_2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [61]:
df = pd.DataFrame(data=[], columns=["Image_name", "Pipeline_GT_IOU", "Wizart_GT_IOU"])

In [63]:
for image in listdir("images/outputs/segmentation/"):
    print(image)
    filename = "images/outputs/segmentation/" + image
    image_name = image[:-4]
    pl = general_methods.import_cv2_image(filename)
    gt = general_methods.import_cv2_image("data-collection/groundtruths/segmentation-resized/" + image_name + "_wall_seg.png")
    pl_iou = get_iuo(gt, pl)

    try:
        wiz = general_methods.import_cv2_image("images/seg-images-resized/" + image_name + ".png")
        wiz_iou = get_iuo(gt, wiz)
    except:
        wiz_iou = float("nan")

    new_data = [image_name, pl_iou, wiz_iou]
    df.loc[len(df)] = new_data
    
    

ADE_val_00000079.jpg
ADE_val_00000085.jpg
ADE_val_00000091.jpg
ADE_val_00000092.jpg
ADE_val_00000093.jpg
ADE_val_00000101.jpg
ADE_val_00000105.jpg
ADE_val_00000119.jpg
ADE_val_00000141.jpg
ADE_val_00000151.jpg
ADE_val_00000160.jpg
ADE_val_00000245.jpg
ADE_val_00000303.jpg
ADE_val_00000314.jpg
ADE_val_00000316.jpg
ADE_val_00000318.jpg
ADE_val_00000420.jpg
ADE_val_00000423.jpg
ADE_val_00000467.jpg
ADE_val_00000474.jpg
ADE_val_00000480.jpg
ADE_val_00000517.jpg
ADE_val_00000522.jpg
ADE_val_00000640.jpg
ADE_val_00001045.jpg
ADE_val_00001081.jpg
ADE_val_00001083.jpg
ADE_val_00001092.jpg
ADE_val_00001097.jpg
ADE_val_00001153.jpg
ADE_val_00001169.jpg
ADE_val_00001312.jpg
ADE_val_00001315.jpg
ADE_val_00001322.jpg
ADE_val_00001478.jpg
ADE_val_00001488.jpg
ADE_val_00001505.jpg
ADE_val_00001518.jpg
ADE_val_00001525.jpg
ADE_val_00001527.jpg
ADE_val_00001531.jpg
ADE_val_00001638.jpg
ADE_val_00001639.jpg
ADE_val_00001950.jpg


In [64]:
df

,Image_name,Pipeline_GT_IOU,Wizart_GT_IOU
0,ADE_val_00000079,0.789079,0.584868
1,ADE_val_00000085,0.880435,0.536162
2,ADE_val_00000091,0.759598,0.597668
3,ADE_val_00000092,0.675897,NaN
4,ADE_val_00000093,0.936711,0.782645
5,ADE_val_00000101,0.329860,0.423798
6,ADE_val_00000105,0.648056,NaN
7,ADE_val_00000119,0.907165,NaN
8,ADE_val_00000141,0.939067,0.834770
9,ADE_val_00000151,0.882443,0.744665


In [ ]:
df.to_csv("data-collection/outputs/segmentation-iou.csv", index=False)